In [1]:
%cd /hdd/yuchen/cs292/StableDiffusionReconstruction/codes/utils

/hdd/yuchen/cs292/StableDiffusionReconstruction/codes/utils


In [2]:
import argparse
import os
import numpy as np
import pandas as pd
from nsd_access import NSDAccess
import scipy.io
import argparse, os
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from tqdm import tqdm
from einops import repeat
from torch import autocast
from contextlib import nullcontext
# from pytorch_lightning import seed_everything
from nsd_access import NSDAccess
from PIL import Image
import scipy.io
import random

In [3]:
%cd /hdd/yuchen/cs292/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion

/hdd/yuchen/cs292/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion


In [4]:
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler

In [5]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/cs292/'
    imgidx = (0, 10)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early 
    
    method = 'cvpr'
    
opt = Opt()

In [6]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

## make_subjmri.py

In [11]:
subject = opt.subject
# atlasname = 'streams'
# atlasname = 'HCP_MMP1'
atlasname = 'Kastner2015'

    
nsda = NSDAccess(opt.root_folder + 'nsd/')
nsd_expdesign = scipy.io.loadmat(opt.root_folder + 'nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

    # Note that most of nsd_expdesign indices are 1-base index!
    # This is why subtracting 1
sharedix = nsd_expdesign['sharedix'] -1 

behs = pd.DataFrame()
for i in range(1,38):
    beh = nsda.read_behavior(subject=subject, 
                                session_index=i)
    behs = pd.concat((behs,beh))

In [12]:
stims_unique = behs['73KID'].unique() - 1
stims_all = behs['73KID'] - 1

savedir = f'{opt.root_folder}/mrifeat/{subject}/'
os.makedirs(savedir, exist_ok=True)

if not os.path.exists(f'{savedir}/{subject}_stims.npy'):
    np.save(f'{savedir}/{subject}_stims.npy',stims_all)
    np.save(f'{savedir}/{subject}_stims_ave.npy',stims_unique)

for i in range(1,38):
    print(i)
    beta_trial = nsda.read_betas(subject=subject, 
                                session_index=i, 
                                trial_index=[], # empty list as index means get all for this session
                                data_type='betas_fithrf_GLMdenoise_RR',
                                data_format='func1pt8mm')
    if i==1:
        betas_all = beta_trial
    else:
        betas_all = np.concatenate((betas_all,beta_trial),0)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [51]:
atlasname = 'Kastner2015'
nsda = NSDAccess(opt.root_folder + 'nsd/')

atlas = nsda.read_atlas_results(subject=subject, atlas=atlasname, data_format='func1pt8mm')

for roi,val in atlas[1].items():
    if val == 0:
        print('SKIP')
        continue
    else:
        betas_roi = betas_all[:,atlas[0].transpose([2,1,0])==val]

    print(roi, betas_roi.shape)
        
    # Averaging for each stimulus
    betas_roi_ave = []
    for stim in stims_unique:
        stim_mean = np.mean(betas_roi[stims_all == stim,:],axis=0)
        betas_roi_ave.append(stim_mean)
    betas_roi_ave = np.stack(betas_roi_ave)
    print(betas_roi_ave.shape)
        
        # Train/Test Split
        # ALLDATA
    betas_tr = []
    betas_te = []

    for idx,stim in enumerate(stims_all):
        if stim in sharedix:
            betas_te.append(betas_roi[idx,:])
        else:
            betas_tr.append(betas_roi[idx,:])

    betas_tr = np.stack(betas_tr)
    betas_te = np.stack(betas_te)    
        
        # AVERAGED DATA        
    betas_ave_tr = []
    betas_ave_te = []
    for idx,stim in enumerate(stims_unique):
        if stim in sharedix:
            betas_ave_te.append(betas_roi_ave[idx,:])
        else:
            betas_ave_tr.append(betas_roi_ave[idx,:])
    betas_ave_tr = np.stack(betas_ave_tr)
    betas_ave_te = np.stack(betas_ave_te)    
        
        # Save
    np.save(f'{savedir}/{subject}_{roi}_betas_tr.npy',betas_tr)
    np.save(f'{savedir}/{subject}_{roi}_betas_te.npy',betas_te)
    np.save(f'{savedir}/{subject}_{roi}_betas_ave_tr.npy',betas_ave_tr)
    np.save(f'{savedir}/{subject}_{roi}_betas_ave_te.npy',betas_ave_te)



/hdd/yuchen/cs292/nsd/nsddata/freesurfer/fsaverage/label/Kastner2015.mgz.ctab
SKIP
V1v (27750, 1229)
(9841, 1229)
V1d (27750, 1045)
(9841, 1045)
V2v (27750, 1054)
(9841, 1054)
V2d (27750, 628)
(9841, 628)
V3v (27750, 763)
(9841, 763)
V3d (27750, 687)
(9841, 687)
hV4 (27750, 473)
(9841, 473)
VO1 (27750, 216)
(9841, 216)
VO2 (27750, 313)
(9841, 313)
PHC1 (27750, 199)
(9841, 199)
PHC2 (27750, 183)
(9841, 183)
TO2 (27750, 55)
(9841, 55)
TO1 (27750, 284)
(9841, 284)
LO2 (27750, 193)
(9841, 193)
LO1 (27750, 343)
(9841, 343)
V3B (27750, 315)
(9841, 315)
V3A (27750, 696)
(9841, 696)
IPS0 (27750, 606)
(9841, 606)
IPS1 (27750, 462)
(9841, 462)
IPS2 (27750, 500)
(9841, 500)
IPS3 (27750, 508)
(9841, 508)
IPS4 (27750, 65)
(9841, 65)
IPS5 (27750, 14)
(9841, 14)
SPL1 (27750, 164)
(9841, 164)
FEF (27750, 72)
(9841, 72)


In [31]:
subject = 'subj01'
roi = 'V1'
ct = 0
for roi in ['V1', 'V2', 'V3', 'V4']:
    savedir = f'{opt.root_folder}/mrifeat/{subject}/'
    ct += np.load(f'{savedir}/{subject}_{roi}_betas_tr.npy').shape[1]

In [32]:
ct

10631

In [27]:
print(np.load(f'{savedir}/{subject}_early_betas_tr.npy').shape)
print(np.load(f'{savedir}/{subject}_ventral_betas_tr.npy').shape)


(24980, 5917)
(24980, 7604)


## img2feat_sd1.py

In [5]:
def load_model_from_config(config, ckpt, gpu, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda(f"cuda:{gpu}")
    model.eval()
    return model

def load_img_from_arr(img_arr,resolution):
    image = Image.fromarray(img_arr).convert("RGB")
    w, h = resolution, resolution
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [6]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    imgidx = (0, 73000)
    gpu = 1
    seed = 42
    
opt = Opt()

In [7]:
# seed_everything(opt.seed)
imgidx = opt.imgidx
gpu = opt.gpu
resolution = 320
batch_size = 1
ddim_steps = 50
ddim_eta = 0.0
strength = 0.8
scale = 5.0
nsda = NSDAccess(opt.root_folder + 'nsd/')
config = opt.root_folder + 'StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/configs/stable-diffusion/v1-inference.yaml'
ckpt = opt.root_folder + 'StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt'
config = OmegaConf.load(f"{config}")
torch.cuda.set_device(gpu)
os.makedirs(f'{opt.root_folder}nsdfeat/init_latent/', exist_ok=True)
os.makedirs(f'{opt.root_folder}nsdfeat/c/', exist_ok=True)

In [8]:
# Load moodels
precision = 'autocast'
precision_scope = autocast if precision == "autocast" else nullcontext
model = load_model_from_config(config, f"{ckpt}", gpu)
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
sampler = DDIMSampler(model)
sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(strength * ddim_steps)
print(f"target t_enc is {t_enc} steps")

Loading model from /hdd/yuchen/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode


/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.23.self_attn.out_proj.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.12.self_attn.out_proj.weight', 'vision_model.encoder.layers.23.self_attn.k_proj.bias', 'vision_model.encoder.layers.4.layer_norm2.bias', 'vision_model.encoder.layers.20.self_attn.v_proj.bias', 'vision_model.encoder.layers.19.mlp.fc1.weight', 'vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.encoder.layers.13.self_attn.v_proj.bias', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.15.layer_norm1.weight', 'vision_model.encoder.layers.21.self_attn.q_proj.bias', 'vision_model.encoder.layers.11.self_attn.k_proj.weight', 'vision_model.encoder.layers.19.mlp.fc1.bias', 'vision_model.encoder.layers.14.layer_norm2.bias', 'vision_model.encod

target t_enc is 40 steps


In [14]:
init_latent.max(), init_latent.min()

(tensor(3.9335, device='cuda:1'), tensor(-3.5237, device='cuda:1'))

In [9]:
# Sample
for s in tqdm(range(imgidx[0],imgidx[1])):
    print(f"Now processing image {s:06}")
    prompt = []
    prompts = nsda.read_image_coco_info([s],info_type='captions')
    for p in prompts:
        prompt.append(p['caption'])    
        
    img = nsda.read_images(s)
    init_image = load_img_from_arr(img,resolution).to(device)
    init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

    
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                uc = model.get_learned_conditioning(batch_size * [""])
                c = model.get_learned_conditioning(prompt).mean(axis=0).unsqueeze(0)
    
                        # encode (scaled latent)
                z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        # decode it
                samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                                                unconditional_conditioning=uc,)
                        
    init_latent = init_latent.cpu().detach().numpy().flatten()
    c = c.cpu().detach().numpy().flatten()
    np.save(f'{opt.root_folder}nsdfeat/init_latent/{s:06}.npy',init_latent)
    np.save(f'{opt.root_folder}nsdfeat/c/{s:06}.npy',c)



  0%|                                                 | 0/73000 [00:00<?, ?it/s]

Now processing image 000000
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


  0%|                                                 | 0/73000 [00:00<?, ?it/s]


## make_subjstim.py

In [8]:
opt.subject

'subj01'

In [17]:
opt.featname = 'init_latent'
opt.use_stim = 'each'

subject= opt.subject
use_stim = opt.use_stim
featname = opt.featname
topdir = opt.root_folder + 'nsdfeat/'
savedir = f'{topdir}/subjfeat/'
featdir = f'{topdir}/{featname}/'

nsd_expdesign = scipy.io.loadmat(opt.root_folder + 'nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

    # Note that most of them are 1-base index!
    # This is why I subtract 1
sharedix = nsd_expdesign['sharedix'] -1 

if use_stim == 'ave':
    stims = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_ave.npy')
else: # Each
    stims = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims.npy')
    

In [18]:
feats = []
tr_idx = np.zeros(len(stims))

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    feat = np.load(f'{featdir}/{s:06}.npy')
    feats.append(feat)

feats = np.stack(feats)    

os.makedirs(savedir, exist_ok=True)

feats_tr = feats[tr_idx==1,:]
feats_te = feats[tr_idx==0,:]
np.save(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_tridx.npy',tr_idx)
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_tr.npy',feats_tr)
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_te.npy',feats_te)

27750it [00:05, 4880.60it/s]


## ridge.py

In [17]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    imgidx = (0, 73000)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early   TODO: how do they select that ?
    
opt = Opt()

In [25]:
opt.target = 'init_latent'
opt.roi = 'early'

target = opt.target
roi = opt.roi

backend = set_backend("numpy", on_error="warn")
subject=opt.subject

if target == 'c' or target == 'init_latent': # CVPR
    alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]
else: # text / GAN / depth decoding (with much larger number of voxels)
    alpha = [10000, 20000, 40000]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    
mridir = f'{opt.root_folder}mrifeat/{subject}'
featdir = f'{opt.root_folder}nsdfeat/subjfeat'
savedir = f'{opt.root_folder}decoded/{subject}/'
os.makedirs(savedir, exist_ok=True)

In [26]:
X = []
X_te = []
for croi in [roi]:
    if 'conv' in target: # We use averaged features for GAN due to large number of dimension of features
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32")
    else:
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_tr.npy').astype("float32")
    cX_te = np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32")
    X.append(cX)
    X_te.append(cX_te)
X = np.hstack(X)
X_te = np.hstack(X_te)
    
Y = np.load(f'{featdir}/{subject}_each_{target}_tr.npy').astype("float32").reshape([X.shape[0],-1])
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32").reshape([X_te.shape[0],-1])
    
print(f'Now making decoding model for... {subject}:  {roi}, {target}')
print(f'X {X.shape}, Y {Y.shape}, X_te {X_te.shape}, Y_te {Y_te.shape}')
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
rs = correlation_score(Y_te.T,scores.T)
print(f'Prediction accuracy is: {np.mean(rs):3.3}')

np.save(f'{savedir}/{subject}_{"_".join(roi)}_scores_{target}.npy',scores)

Now making decoding model for... subj01:  early, init_latent
X (24980, 5917), Y (24980, 6400), X_te (982, 5917), Y_te (982, 6400)
Prediction accuracy is: 0.239


In [10]:
target  = 'c'
subject = 'subj01'
featdir = '/hdd/yuchen/nsdfeat/subjfeat'
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32")
scores = np.load('/hdd/yuchen/decoded/subj01/subj01_v_e_n_t_r_a_l_scores_c.npy')

from sklearn.metrics import mean_squared_error
mean_squared_error(Y_te,scores)
# mse for text

0.6907157

In [30]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_te,scores)
# mse for image

0.61325544

## diffusion_decoding.py 

In [7]:
import h5py
from PIL import Image
import scipy.io
import argparse, os
import pandas as pd
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from tqdm import trange
from einops import rearrange
from torch import autocast
from contextlib import nullcontext
# from pytorch_lightning import seed_everything
import sys
sys.path.append("../utils/")
from nsd_access.nsda import NSDAccess
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler

In [8]:
def load_model_from_config(config, ckpt, gpu, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda(f"cuda:{gpu}")
    model.eval()
    return model

def load_img_from_arr(img_arr):
    image = Image.fromarray(img_arr).convert("RGB")
    w, h = 512, 512
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [9]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/cs292/'
    imgidx = (0, 10)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early 
    
    method = 'cvpr'
    
opt = Opt()

In [10]:
imgidx = opt.imgidx
gpu = opt.gpu
method = opt.method
subject=opt.subject
torch.cuda.set_device(opt.gpu)

captdir = f'{opt.root_folder}decoded/{subject}/captions/'

    # Load NSD information
nsd_expdesign = scipy.io.loadmat(f'{opt.root_folder}nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

    # Note that mos of them are 1-base index!
    # This is why I subtract 1
sharedix = nsd_expdesign['sharedix'] -1 

nsda = NSDAccess(f'{opt.root_folder}nsd/')
sf = h5py.File(nsda.stimuli_file, 'r')
sdataset = sf.get('imgBrick')

stims_ave = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_ave.npy')

In [11]:
tr_idx = np.zeros_like(stims_ave)
for idx, s in enumerate(stims_ave):
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1

In [12]:
    # Load Stable Diffusion Model
path = '/hdd/yuchen/cs292/StableDiffusionReconstruction/codes/diffusion_sd1'
config = path + '/stable-diffusion/configs/stable-diffusion/v1-inference.yaml'
ckpt = path + '/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt'
config = OmegaConf.load(f"{config}")
torch.cuda.set_device(gpu)
model = load_model_from_config(config, f"{ckpt}", gpu)

n_samples = 1
ddim_steps = 50
ddim_eta = 0.0
strength = 0.8
scale = 5.0
n_iter = 5
precision = 'autocast'
precision_scope = autocast if precision == "autocast" else nullcontext
batch_size = n_samples
device = torch.device(f"cuda:{opt.gpu}") if torch.cuda.is_available() else torch.device("cpu")

Loading model from /hdd/yuchen/cs292/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode


/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.14.self_attn.v_proj.bias', 'vision_model.encoder.layers.23.layer_norm2.weight', 'vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.encoder.layers.3.layer_norm2.bias', 'vision_model.encoder.layers.17.self_attn.out_proj.bias', 'vision_model.encoder.layers.10.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.self_attn.v_proj.bias', 'vision_model.encoder.layers.7.self_attn.k_proj.bias', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.weight', 'vision_model.encoder.layers.9.self_attn.v_proj.bias', 'vision_model.encoder.layers.22.layer_norm2.weight', 'vision_model.encoder.layers.23.mlp.fc1.bias', 'vision_model.encoder.layers.10.self_attn.v_proj.weight', 'vision_model.encoder.layers.18.mlp.fc2.weight', 'vision_model.

In [13]:
outdir = f'{opt.root_folder}decoded/image-{method}/{subject}/'
os.makedirs(outdir, exist_ok=True)

sample_path = os.path.join(outdir, f"samples")
os.makedirs(sample_path, exist_ok=True)
precision = 'autocast'
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
sampler = DDIMSampler(model)

sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)

assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(strength * ddim_steps)
print(f"target t_enc is {t_enc} steps")


target t_enc is 40 steps


In [ ]:
%%capture
scores_c = np.load('/hdd/yuchen/cs292/temp_lst_latent_c_roi.npy')
scores_latent = np.load('/hdd/yuchen/cs292/temp_lst_latent_init_roi.npy')
sample_path = '/hdd/yuchen/cs292/decoded/image-cvpr/subj01/nsd_roi/samples/'

imgidx_te = np.where(tr_idx==0)[0]
# ct = 0
num_test_img = imgidx_te.shape[0]

for imgidx in range(num_test_img):
        
# Load z (Image)
    imgidx_te = np.where(tr_idx==0)[0][imgidx] # Extract test image index
    idx73k= stims_ave[imgidx_te]
    Image.fromarray(np.squeeze(sdataset[idx73k,:,:,:]).astype(np.uint8)).save(
        os.path.join(sample_path, f"{imgidx:05}_org.png"))    
        
    if method in ['cvpr','text']:
        # roi_latent = 'early'
        # roi_latent = 'e_a_r_l_y'
        
        # scores_latent = np.load(f'{opt.root_folder}decoded/{subject}/{subject}_{roi_latent}_scores_init_latent.npy')
        imgarr = torch.Tensor(scores_latent[imgidx,:].reshape(4,40,40)).unsqueeze(0).to(f'cuda:{opt.gpu}')
    
        # Generate image from Z
        precision_scope = autocast if precision == "autocast" else nullcontext
        with torch.no_grad():
            with precision_scope("cuda"):
                with model.ema_scope():
                    x_samples = model.decode_first_stage(imgarr)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
        im = Image.fromarray(x_sample.astype(np.uint8)).resize((512,512))
        im = np.array(im)
    
    init_image = load_img_from_arr(im).to(f'cuda:{opt.gpu}')
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
    
    # Load c (Semantics)
    if method == 'cvpr':
        # roi_c = 'ventral'
        # roi_c = 'v_e_n_t_r_a_l'
        # scores_c = np.load(f'{opt.root_folder}decoded/{subject}/{subject}_{roi_c}_scores_c.npy')
        # carr = scores_c[imgidx,:].reshape(77,768)
        carr = scores_c[imgidx,:]
        c = torch.Tensor(carr).unsqueeze(0).to(f'cuda:{opt.gpu}')
    
        # Generate image from Z (image) + C (semantics)
    base_count = 0
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for n in trange(n_iter, desc="Sampling"):
                    uc = model.get_learned_conditioning(batch_size * [""])
    
                        # encode (scaled latent)
                    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        # decode it
                    samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                                                unconditional_conditioning=uc,)
    
                    x_samples = model.decode_first_stage(samples)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    Image.fromarray(x_sample.astype(np.uint8)).save(
                        os.path.join(sample_path, f"{imgidx:05}_{base_count:03}.png"))    
                    base_count += 1
    # if imgidx == 100:
    #     break


In [ ]:
!pkill tmux